In [36]:
import numpy as np
import pandas as pd
from itertools import zip_longest

In [2]:
train = pd.read_hdf('data/ready_train.hdf')

In [59]:
def binarize_feature(data, width):
    bin_repr = pd.DataFrame(data=np.zeros((len(data),width), dtype=np.bool_))
    for ind, e in enumerate(data):
        binr = np.binary_repr(e, width)
        for i in range(width):
            bin_repr.at[ind,i] = int(binr[i])
    return bin_repr
#--------------------------------------------------------------
train_bin = pd.DataFrame(index=range(len(train)))


In [56]:
%%time

bin_repr = binarize_feature(train.channelGrouping, 3)
train_bin = pd.concat((train_bin, bin_repr), axis=1)

CPU times: user 1min 57s, sys: 86 ms, total: 1min 57s
Wall time: 1min 59s


In [61]:
%%time
bin_repr = binarize_feature(train.medium, 3)
train_bin = pd.concat((train_bin, bin_repr), axis=1)


CPU times: user 1min 38s, sys: 7.26 ms, total: 1min 38s
Wall time: 1min 39s


In [63]:
%%time
bin_repr = binarize_feature(train.continent, 3)
train_bin = pd.concat((train_bin, bin_repr), axis=1)


CPU times: user 1min 39s, sys: 32.9 ms, total: 1min 39s
Wall time: 1min 41s


In [64]:
%%time
bin_repr = binarize_feature(train.subContinent, 5)
train_bin = pd.concat((train_bin, bin_repr), axis=1)


CPU times: user 2min 42s, sys: 42.1 ms, total: 2min 42s
Wall time: 2min 44s


In [65]:
%%time
bin_repr = binarize_feature(train.browser, 4)
train_bin = pd.concat((train_bin, bin_repr), axis=1)


CPU times: user 2min 14s, sys: 76.7 ms, total: 2min 14s
Wall time: 2min 15s


In [66]:
%%time
bin_repr = binarize_feature(train.operatingSystem, 5)
train_bin = pd.concat((train_bin, bin_repr), axis=1)


CPU times: user 2min 40s, sys: 90.6 ms, total: 2min 40s
Wall time: 2min 42s


In [67]:
%%time
bin_repr = binarize_feature(train.deviceCategory, 2)
train_bin = pd.concat((train_bin, bin_repr), axis=1)


CPU times: user 1min 11s, sys: 87.9 ms, total: 1min 11s
Wall time: 1min 12s


In [86]:
%%time
bin_repr = binarize_feature(train.country, 8)
train_bin = pd.concat((train_bin, bin_repr), axis=1)


CPU times: user 4min 34s, sys: 255 ms, total: 4min 34s
Wall time: 4min 40s


-------------------

In [89]:
# checkpoint 1
train_bin.columns = range(len(train_bin.columns))
train_bin.to_hdf('./data/ready_train_bin.hdf', key='binarizing1')

In [97]:
# fix train.year from [ 2017 - 2016] to [1, 0]
train.year = train.year.apply(lambda d : 1 if d == 2017 else 0 )
train.visitNumber = np.log1p(train.visitNumber)
train.hits = np.log1p(train.hits)
train.pageviews = np.log1p(train.pageviews)
train.dayOfYear = np.log1p(train.dayOfYear)
train.target = np.log1p(train.target)

train.to_hdf('./data/ready_train_fixed.hdf', key='fixed_year')

In [81]:
# Merge binary features aswell
train_bin = pd.concat((train_bin, train.bounces.astype(np.bool_)), axis=1)
train_bin = pd.concat((train_bin, train.newVisits.astype(np.bool_)), axis=1)
train_bin = pd.concat((train_bin, train.isTrueDirect.astype(np.bool_)), axis=1)
train_bin = pd.concat((train_bin, train.isVideoAd.astype(np.bool_)), axis=1)
train_bin = pd.concat((train_bin, train.year.astype(np.bool_)), axis=1)
#---------------------------
# checkpoint 2
train_bin.columns = range(len(train_bin.columns))
train_bin.to_hdf('./data/ready_train_bin.hdf', key='binarizing2')

In [90]:
# Merge other integer columns
train_bin = pd.concat((train_bin, train.visitNumber), axis=1)
train_bin = pd.concat((train_bin, train.hits), axis=1)
train_bin = pd.concat((train_bin, train.pageviews), axis=1)
train_bin = pd.concat((train_bin, train.dayOfYear), axis=1)
#---------------------------
# checkpoint 3
train_bin.columns = range(len(train_bin.columns))
train_bin.to_hdf('./data/ready_train_bin.hdf', key='binarizing3')

In [92]:
# last step, add the target :
train_bin = pd.concat((train_bin, train.target), axis=1)
#-------------------------------
train_bin.rename(columns={'target':-1}, inplace=True)
# checkpoint 4 # last
train_bin.to_hdf('./data/ready_train_bin.hdf', key='last')

In [96]:
train_bin.shape

(903653, 43)

---------